In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from numpy import asarray
import glob
import cv2
import tensorflow as tf
from tensorflow import keras

Import Model

In [11]:
UnWeighted_Model=keras.models.load_model('drive/MyDrive/CNNSpeciesAcc0.9133val0.8493')

Import Data

In [3]:
X_train=np.load('drive/MyDrive/X_train_strat.npy')

In [4]:
y_train=np.load('drive/MyDrive/y_train_strat.npy')

In [5]:
X_val=np.load('drive/MyDrive/X_val_strat.npy')

In [6]:
y_val=np.load('drive/MyDrive/y_val_strat.npy')

Get val data probabilities

In [12]:
y_proba=UnWeighted_Model.predict(X_val)

In [13]:
y_proba[1]

array([3.6509544e-01, 5.0902392e-05, 1.5057314e-02, 1.4492625e-06,
       8.8719971e-13, 2.0239108e-06, 1.9344385e-04, 7.9907929e-12,
       3.2738722e-06, 3.8893751e-05, 6.2414634e-22, 9.8531291e-07,
       5.1459068e-01, 1.1715752e-02, 5.5191405e-05, 5.8600608e-10,
       2.8074014e-06, 9.1593608e-04, 1.9018884e-06, 6.6723888e-10,
       1.2768989e-14, 6.2570500e-12, 1.8522213e-08, 1.6897484e-07,
       9.2255339e-02, 1.5063314e-07, 3.5252967e-09, 1.8258364e-05],
      dtype=float32)

In [14]:
# Turn one hot array into integers
y_val_1d=np.argmax(y_val, axis=1)
y_predictions=np.argmax(y_proba, axis=1)

In [15]:
y_dataframe=pd.DataFrame({'y_predictions': y_predictions,
                          'y_actual': y_val_1d
                          })

In [16]:
y_dataframe

,y_predictions,y_actual
0,25,25
1,12,12
2,2,2
3,23,23
4,9,14
...,...,...
8268,1,1
8269,13,13
8270,1,1
8271,1,1


In [17]:
y_proba_list=[]

for each in y_proba:
  y_proba_list.append(each)

In [18]:
y_dataframe['y_proba']=y_proba_list

In [19]:
#round to 5 decimals

rounded_proba_list=[]
for each_array in y_proba_list:
  new_array=[]
  for each_number in each_array:
    rounded_number=round(each_number, ndigits=5)
    new_array.append('{0:6f}'.format(rounded_number))
  rounded_proba_list.append(new_array)

In [20]:
y_dataframe['y_rounded_proba']=rounded_proba_list

In [21]:
y_dataframe.head(20)

,y_predictions,y_actual,y_proba,y_rounded_proba
0,25,25,"[1.0411376e-07, 1.8620929e-06, 0.053617027, 3....","[0.000000, 0.000000, 0.053620, 0.000000, 0.000..."
1,12,12,"[0.36509544, 5.0902392e-05, 0.015057314, 1.449...","[0.365100, 0.000050, 0.015060, 0.000000, 0.000..."
2,2,2,"[2.6952187e-11, 5.936212e-09, 0.84236664, 2.76...","[0.000000, 0.000000, 0.842370, 0.000000, 0.000..."
3,23,23,"[1.4206581e-22, 4.5429557e-20, 2.0954663e-16, ...","[0.000000, 0.000000, 0.000000, 0.000000, 0.000..."
4,9,14,"[0.010332668, 0.00021841915, 0.0023903393, 0.0...","[0.010330, 0.000220, 0.002390, 0.050820, 0.000..."
5,7,7,"[1.6813204e-14, 3.618786e-15, 1.5613007e-10, 0...","[0.000000, 0.000000, 0.000000, 0.000000, 0.000..."
6,0,13,"[0.59035474, 0.00045025913, 0.08361771, 0.0001...","[0.590350, 0.000450, 0.083620, 0.000120, 0.000..."
7,2,2,"[9.644166e-14, 2.007716e-11, 0.9982496, 7.7603...","[0.000000, 0.000000, 0.998250, 0.000000, 0.000..."
8,13,12,"[0.011090288, 0.00071977166, 1.3795623e-05, 1....","[0.011090, 0.000720, 0.000010, 0.000000, 0.000..."
9,2,2,"[1.312381e-13, 2.8090893e-12, 0.9999609, 4.811...","[0.000000, 0.000000, 0.999960, 0.000000, 0.000..."


In [22]:
y_dataframe['y_rounded_proba'][15]

['0.000000',
 '0.000000',
 '0.952800',
 '0.000000',
 '0.000000',
 '0.000000',
 '0.000000',
 '0.000000',
 '0.006710',
 '0.000000',
 '0.000000',
 '0.000100',
 '0.000000',
 '0.000000',
 '0.000000',
 '0.000000',
 '0.038260',
 '0.000000',
 '0.000000',
 '0.000130',
 '0.000000',
 '0.000000',
 '0.000010',
 '0.000000',
 '0.000000',
 '0.001980',
 '0.000000',
 '0.000000']

In [ ]:
## locate incorrectly classified animals


Create Weights for underepresented classes

In [29]:
from sklearn.utils import class_weight

In [30]:
# Turn one hot array into integers
y_train_1d=np.argmax(y_train, axis=1)

class_weights=class_weight.compute_class_weight(class_weight='balanced',
                                                classes=np.unique(y_train_1d),
                                                y=y_train_1d)

In [31]:
class_weights

array([  0.24477009,   0.37731481,   0.16976727,  11.8175    ,
        19.69583333,   5.13804348,   5.54812207,   0.57674475,
         0.54035208,   1.3837822 , 131.30555556,  15.96959459,
         1.61001362,   0.24650605,   0.75366709,   7.47943038,
         1.07334242,   1.12978011,  12.18298969,   7.03422619,
        23.635     ,  31.09868421,   4.3129562 ,   4.96533613,
         2.09530142,   1.08021024,   3.69296875,   7.9847973 ])

weights array needs to be dictionary

In [32]:
class_weight_dict=dict(enumerate(class_weights))

In [33]:
class_weight_dict

{0: 0.24477009113504558,
 1: 0.3773148148148148,
 2: 0.16976727481683665,
 3: 11.8175,
 4: 19.695833333333333,
 5: 5.13804347826087,
 6: 5.5481220657277,
 7: 0.5767447535383113,
 8: 0.5403520804755373,
 9: 1.3837822014051522,
 10: 131.30555555555554,
 11: 15.969594594594595,
 12: 1.6100136239782017,
 13: 0.2465060492282019,
 14: 0.7536670918367347,
 15: 7.4794303797468356,
 16: 1.0733424159854676,
 17: 1.1297801147227533,
 18: 12.18298969072165,
 19: 7.034226190476191,
 20: 23.635,
 21: 31.098684210526315,
 22: 4.312956204379562,
 23: 4.965336134453781,
 24: 2.0953014184397163,
 25: 1.0802102376599634,
 26: 3.69296875,
 27: 7.984797297297297}

In [7]:
#Manually adjusting weights for only the poorly classified classes
adjusted_class_weights={
    0: 1.,
    1: 1.,
    2: 1.,
    3: 1.,
    4: 1.,
    5: 1.,
    6: 5.,
    7: 1.,
    8: 1.,
    9: 1.,
    10: 1.,
    11: 6.,
    12: 1.,
    13: 1.,
    14: 1.,
    15: 4.,
    16: 1.,
    17: 1.,
    18: 1.,
    19: 4.,
    20: 10.,
    21: 10.,
    22: 1.,
    23: 1.,
    24: 1.,
    25: 1.,
    26: 1.,
    27: 1.}

Add weights to training

In [8]:
from tensorflow.keras import layers
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model4 = keras.models.Sequential()
model4.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(150,200,1),padding='same'))
model4.add(MaxPooling2D((2, 2),padding='same'))
model4.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.28))
model4.add(Conv2D(128, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.35))
model4.add(Conv2D(256, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model4.add(Dropout(0.40))
model4.add(Conv2D(512, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model4.add(Dropout(0.40))
model4.add(Conv2D(800, (3, 3), activation='relu',padding='same'))                 
model4.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model4.add(Dropout(0.40))
model4.add(Flatten())
model4.add(Dense(600, activation='relu'))               
model4.add(Dense(28, activation='softmax'))

In [9]:
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model4.fit(X_train, y_train, class_weight=adjusted_class_weights, epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
1035/1035 [==============================] - 71s 53ms/step - loss: 2.4223 - accuracy: 0.3773 - val_loss: 1.5480 - val_accuracy: 0.5206
Epoch 2/30
1035/1035 [==============================] - 50s 49ms/step - loss: 1.7234 - accuracy: 0.5613 - val_loss: 1.3672 - val_accuracy: 0.5833
Epoch 3/30
1035/1035 [==============================] - 50s 48ms/step - loss: 1.5369 - accuracy: 0.6010 - val_loss: 1.2107 - val_accuracy: 0.6325
Epoch 4/30
1035/1035 [==============================] - 50s 48ms/step - loss: 1.4609 - accuracy: 0.6194 - val_loss: 1.2531 - val_accuracy: 0.6167
Epoch 5/30
1035/1035 [==============================] - 50s 48ms/step - loss: 1.3679 - accuracy: 0.6369 - val_loss: 1.2098 - val_accuracy: 0.6208
Epoch 6/30
1035/1035 [==============================] - 50s 48ms/step - loss: 1.3229 - accuracy: 0.6435 - val_loss: 1.0476 - val_accuracy: 0.6745
Epoch 7/30
1035/1035 [==============================] - 50s 48ms/step - loss: 1.2769 - accuracy: 0.6532 - val_loss: 1.1655 -

KeyboardInterrupt: ignored

In [ ]:
## weighted all 1's   Accuracy: 79   Val Accuracy: 79  after 29 epochs